In [3]:
%pwd

'd:\\SUJITH_\\DataScience\\BAPPY_GenAI\\MedicalChatBotGenAI\\research'

In [30]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings

In [5]:
%cd  ..

d:\SUJITH_\DataScience\BAPPY_GenAI\MedicalChatBotGenAI


In [6]:
#Extract data from pdf file 
def load_pdf_file(file):
    loader=DirectoryLoader(file,        glob='*.pdf',    loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

extracted_data=load_pdf_file(file='data/')


In [7]:
len(extracted_data)

637

In [19]:
type(extracted_data)

list

In [31]:
#Split data into chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks
text_chunks=text_split(extracted_data)

In [21]:
len(text_chunks)

TypeError: object of type 'Document' has no len()

In [25]:
type(text_chunks[0].page_content)
text_chunks[0].page_content

'The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'

In [34]:
#vector embedding
def download_hugging_face_embeddings():
  embedding=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')# 384 diamensional vector - https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
  return embedding
embedding=download_hugging_face_embeddings()


c:\Users\91890\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [38]:
query_result=embedding.embed_query("Hello world suji th")
print(" length of sample vector embedding ",len(query_result))
#query_result

 length of sample vector embedding  384


[-0.04819633811712265,
 0.15012630820274353,
 0.021478408947587013,
 -0.0044724466279149055,
 -0.04288671165704727,
 -0.021710975095629692,
 0.02975575067102909,
 -0.03278796747326851,
 -0.007388236466795206,
 0.01123051904141903,
 0.09345773607492447,
 -0.03344658762216568,
 0.015225041657686234,
 -0.060943469405174255,
 0.11529836803674698,
 -0.027648624032735825,
 -0.014623572118580341,
 0.014753407798707485,
 -0.08076632767915726,
 -0.10180769860744476,
 0.06451240181922913,
 0.09679209440946579,
 -0.0012804794823750854,
 -0.052918437868356705,
 -0.04407312721014023,
 0.009784083813428879,
 0.0390218086540699,
 0.025619475170969963,
 0.1521606147289276,
 -0.013859405182301998,
 -0.042430516332387924,
 0.0005080864066258073,
 0.054551225155591965,
 0.013029119931161404,
 -0.020575538277626038,
 0.07575448602437973,
 0.0007773303077556193,
 -0.028798870742321014,
 0.040226567536592484,
 -0.08949121087789536,
 0.02399080991744995,
 -0.04085077717900276,
 -0.038974881172180176,
 -0.048

In [55]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
from  dotenv import load_dotenv
load_dotenv()


True

In [59]:
PINECONE_API_KEY =os.getenv('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)

In [57]:
index_name = "medical-bot-index"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [61]:
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and  insert the mebedding to your Pinecone index
docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embedding

)

In [62]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and  insert the mebedding to your Pinecone index
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding

)

In [63]:
retriver=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [65]:
retriever_dcos=retriver.invoke('What is herniya')

In [69]:
type(retriever_dcos[0].page_content)

str

In [73]:
from langchain_openai import OpenAI
load_dotenv()
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
llm=OpenAI(temperature=0.4,max_tokens=500)

In [75]:
#RAG
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are teh assistant for question -answering task."
    "Use the following piece of retrieved context to answer"
    " teh questions.I f you dont know the answer , say that you \dont know.Use three sentence maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)



In [76]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriver,question_answer_chain)

In [79]:
response=rag_chain.invoke({"input":"what is hepatitis?"})
print(response["answer"])



Hepatitis is a medical condition that involves inflammation of the liver. It can be caused by various infectious or toxic agents and is characterized by symptoms such as jaundice, fever, liver enlargement, and abdominal pain.


In [81]:
response=rag_chain.invoke({"input":"what is cosine?"})
print(response["answer"])



I don't know.
